### Example MNIST: Predicting numbers from 0 to 9

<br>Carlos Estevez
<br>carlos@estevez.do
<br>https://github.com/carlosAtestevez

In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import copy
from torchvision import datasets,transforms

C:\Users\carlo\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def initialize_parameters_deep(layer_dims):    
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters

def linear_forward(A, W, b):
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    return Z, cache

def sigmoid(Z):
    A = 1 / (1 + np.exp(-Z))
    cache = Z
    return A, cache

def relu(Z):
    A = np.maximum(0, Z)
    cache = Z 
    return A, cache

def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

def sigmoid_backward(dA, cache):
    Z = cache
    s = 1 / (1 + np.exp(-Z))
    dZ = dA * s * (1 - s)
    return dZ

def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)
    return A, cache

def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["b" + str(L)], "sigmoid")
    caches.append(cache)
    
    return AL, caches

def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = (-1/m) * np.sum((np.multiply(Y, np.log(AL))) + (np.multiply(1-Y, np.log(1-AL))))
    cost = np.squeeze(cost)
    return cost

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1] 
    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T, dZ)
    return dA_prev, dW, db

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)

    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) 
    current_cache = caches[L-1]
    dA_prev_temp, dW_temp, db_temp = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    grads["dA" + str(L-1)] = dA_prev_temp
    grads["dW" + str(L)] = dW_temp
    grads["db" + str(L)] = db_temp
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation="relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

def update_parameters(params, grads, learning_rate):
    parameters = copy.deepcopy(params)
    L = len(parameters) // 2

    for l in range(L):
        parameters["W" + str(l + 1)] -= learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l + 1)] -= learning_rate * grads["db" + str(l + 1)]
        
    return parameters

def predict(X, y, parameters):
    m = X.shape[1]
    n = len(parameters) // 2
    p = np.zeros((1, m))
    
    probas, caches = L_model_forward(X, parameters)
    
    for i in range(0, probas.shape[1]):
        p[0, i] = 1 if probas[0, i] > 0.5 else 0
    
    print("Accuracy: " + str(np.sum((p == y) / m)))
    return p

def L_layer_model(X, Y, layers_dims, learning_rate=0.0075, num_iterations=3000, print_cost=False):
    costs = []
    parameters = initialize_parameters_deep(layers_dims)

    for i in range(0, num_iterations):
        AL, caches = L_model_forward(X, parameters)
        cost = compute_cost(AL, Y)
        grads = L_model_backward(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and (i % 50 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [4]:
# Define a transform to convert the images to tensors
transform = transforms.Compose([transforms.ToTensor()])

# Load the MNIST dataset
download_first_time = True  # Set this to True if downloading for the first time
train_data = datasets.MNIST(root="./datasets", download=download_first_time, train=True, transform=transform)
test_data = datasets.MNIST(root="./datasets", download=download_first_time, train=False, transform=transform)

# Convert train and test data to numpy arrays
train_images = train_data.data.numpy()
train_labels = train_data.targets.numpy()
test_images = test_data.data.numpy()
test_labels = test_data.targets.numpy()

# Filter the training data for 9s and non-9s
train_filter = (train_labels == 9) | (train_labels != 9)
train_images_filtered = train_images[train_filter]
train_labels_filtered = np.where(train_labels[train_filter] == 9, 1, 0)

# Filter the test data for 9s and non-9s
test_filter = (test_labels == 9) | (test_labels != 9)
test_images_filtered = test_images[test_filter]
test_labels_filtered = np.where(test_labels[test_filter] == 9, 1, 0)

# Normalize the images (values between 0 and 1)
train_images_filtered = train_images_filtered / 255.0
test_images_filtered = test_images_filtered / 255.0

# Flatten the images to shape (28 * 28, num_samples)
train_images_flatten = train_images_filtered.reshape(train_images_filtered.shape[0], -1).T
test_images_flatten = test_images_filtered.reshape(test_images_filtered.shape[0], -1).T

# Reshape the labels to (1, num_samples)
train_labels_flatten = train_labels_filtered.reshape(1, train_labels_filtered.shape[0])
test_labels_flatten = test_labels_filtered.reshape(1, test_labels_filtered.shape[0])

print("Flattened train images shape:", train_images_flatten.shape)
print("Binary train labels shape:", train_labels_flatten.shape)
print("Flattened test images shape:", test_images_flatten.shape)
print("Binary test labels shape:", test_labels_flatten.shape)

train_x = train_images_flatten
train_y = train_labels_flatten

Flattened train images shape: (784, 60000)
Binary train labels shape: (1, 60000)
Flattened test images shape: (784, 10000)
Binary test labels shape: (1, 10000)


In [5]:
input_layer_units = 784
layers_dims = [input_layer_units, 20, 15, 8, 1] #  4-layer model
layers_dims = [input_layer_units, 40, 40, 10, 1] #  4-layer model
#parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 1, print_cost = False)
#parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)
parameters, costs = L_layer_model(train_x, train_y,layers_dims,learning_rate=0.01,num_iterations=1000,print_cost = True)

Cost after iteration 0: 0.6931485946261216
Cost after iteration 50: 0.6219504731652513
Cost after iteration 100: 0.5663806115990968
Cost after iteration 150: 0.5227780224174594
Cost after iteration 200: 0.48831518869351337
Cost after iteration 250: 0.46084878113574357
Cost after iteration 300: 0.4387678266033037
Cost after iteration 350: 0.42086358411281205
Cost after iteration 400: 0.4062260040385699
Cost after iteration 450: 0.3941657626230962
Cost after iteration 500: 0.384156774410162
Cost after iteration 550: 0.3757941899685625
Cost after iteration 600: 0.3687637695846708
Cost after iteration 650: 0.3628195025920561
Cost after iteration 700: 0.35776717186439694
Cost after iteration 750: 0.3534522056887705
Cost after iteration 800: 0.3497506324157066
Cost after iteration 850: 0.3465622924781813
Cost after iteration 900: 0.3438057032391721
Cost after iteration 950: 0.34141414330773423
Cost after iteration 999: 0.3393715424524229
